## Load Libraries

In [1]:
import tqdm
import os
import cv2
import imghdr
import image_to_numpy
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from datetime import datetime
import itertools
from numpy.random import default_rng

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import RandomFlip, RandomRotation, RandomContrast, RandomBrightness, Rescaling, Resizing
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, ReLU
from tensorflow.keras.metrics import F1Score, Precision, Recall, CategoricalAccuracy
from tensorboard.plugins.hparams import api as hp
import sklearn
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.model_selection import StratifiedKFold

from time import perf_counter
import math

import neptune
from neptune.integrations.tensorflow_keras import NeptuneCallback

## Rough Code

In [ ]:
def viz_grid(output, ncols=4, nrows=4):
    fig, ax = plt.subplots(ncols=ncols, nrows=nrows, figsize=(10,10))
    # itertools.product creates a permutation 
    for idx, ax_idx in enumerate(list(itertools.product(range(ncols), range(nrows)))):
        ax[ax_idx[1], ax_idx[0]].imshow(output[0,:,:,idx])

In [ ]:
sm = tf.keras.models.load_model("C:\\Users\\user\\Documents\\Machine-Learning-Hub-1\\Tensorflow\\kaggle_converted\\Final-B4_0_1-converted.keras")

In [ ]:
layers_c1 = sm.layers[:6]
model_c1 = Sequential(layers_c1)

In [ ]:
img_sample_dir = "D:\DATASETS\Tricycle.JPG"
image_size = 256
img_sample = image_to_numpy.load_image_file(os.path.join(img_sample_dir))
img_sample = cv2.resize(img_sample, (image_size, image_size))
img_sample = img_sample.astype(np.float32) / 255.0
fig, ax = plt.subplots(figsize=(3,3))
ax.imshow(img_sample.astype(float))

In [ ]:
output_c1 = model_c1.predict(np.expand_dims(img_sample, 0))

In [ ]:
viz_grid(output_c1, 4, 4)

In [ ]:
layers_c2 = sm.layers[:10]
model_c2 = Sequential(layers_c2)

In [ ]:
output_c2 = model_c2.predict(np.expand_dims(img_sample, 0))
viz_grid(output_c2, 4, 4)

In [ ]:
layers_c3 = sm.layers[:14]
model_c3 = Sequential(layers_c3)

In [ ]:
output_c3 = model_c3.predict(np.expand_dims(img_sample, 0))
fig, ax = plt.subplots(ncols=4, nrows=8, figsize=(15,15))
# itertools.product creates a permutation 
for idx, ax_idx in enumerate(list(itertools.product(range(4), range(8)))):
    ax[ax_idx[1], ax_idx[0]].imshow(output_c3[0,:,:,idx])
    
fig.tight_layout()


## Function for Feature Map Overlay to Image Input

Feature maps used before were numbers 9, 11, and 14!

In [2]:
def fm_to_image_overlay(feature_map, num="0"):
    # Show initial feature map
    plt.figure(figsize=(2,2))
    plt.title(f"Original Feature Map {num}")
    plt.imshow(feature_map)
    fm_to_scale = feature_map
    
    # Set size of feature map to be like input
    orig_img_size = (256,256)
    enlarged = cv2.resize(fm_to_scale, orig_img_size, interpolation=cv2.INTER_LINEAR)
    plt.figure(figsize=(3,3))
    plt.title(f"Enalrged Feature Map {num}")
    plt.imshow(enlarged)
    
    # Set filtering of mask to top 10% of activations
    percentile_90 = np.percentile(enlarged, 90)
    
    # Apply masking
    mask_filter = enlarged >= percentile_90
    output = enlarged.copy()
    output[mask_filter] = 1
    output[~mask_filter] = 0
    plt.figure(figsize=(3,3))
    plt.title(f"Masked Feature Map {num}")
    plt.imshow(output)
    
    # Perform Overlay
    plt.figure(figsize=(3,3))
    plt.title(f"Masked Overlay {num} to Input Image")
    plt.imshow(img_sample)
    plt.imshow(output, alpha=0.25, cmap='cool') 
    # plt.show()

## Convert Rough Code to Functions

In [3]:
def plot_input(img_path, title=""):
    img_sample_dir = img_path
    image_size = 256
    img_sample = image_to_numpy.load_image_file(os.path.join(img_sample_dir))
    img_sample = cv2.resize(img_sample, (image_size, image_size))
    img_sample = img_sample.astype(np.float32) / 255.0
    fig, ax = plt.subplots(figsize=(3,3))
    ax.imshow(img_sample.astype(float))
    ax.axis("off")
    fig.suptitle(title)

def new_viz_grid(output, ncols=4, nrows=4, fig_size=(10,10), title=""):
    fig, ax = plt.subplots(ncols=ncols, nrows=nrows, figsize=fig_size)
    fig.suptitle(title)
    # itertools.product creates a permutation 
    for idx, ax_idx in enumerate(list(itertools.product(range(ncols), range(nrows)))):
        ax[ax_idx[1], ax_idx[0]].imshow(output[0,:,:,idx])
        ax[ax_idx[1], ax_idx[0]].axis("off")
    fig.subplots_adjust(top=0.95, wspace=0)
    
    

In [4]:
def fm_viz(
    sm,
    img_path="D:\DATASETS\Tricycle.JPG",
):
    
    plot_input(img_path, "Input Image")
    
    
    layers_c1 = sm.layers[:6]
    model_c1 = Sequential(layers_c1)
    
    output_c1 = model_c1.predict(np.expand_dims(img_sample, 0))
    new_viz_grid(output_c1, 4, 4, (10,10), "Conv_Set_1's Feature Maps")

    layers_c2 = sm.layers[:10]
    model_c2 = Sequential(layers_c2)
    output_c2 = model_c2.predict(np.expand_dims(img_sample, 0))
    new_viz_grid(output_c2, 4, 4, (10,10), "Conv_Set_2's Feature Maps")

    layers_c3 = sm.layers[:14]
    model_c3 = Sequential(layers_c3)
    output_c3 = model_c3.predict(np.expand_dims(img_sample, 0))
    new_viz_grid(output_c3, 4, 8, (15,15), "Conv_Set_3's Feature Maps")

    fm_to_image_overlay(output_c3[0,:,:,9], "9")
    fm_to_image_overlay(output_c3[0,:,:,11], "11")
    fm_to_image_overlay(output_c3[0,:,:,14], "14")


# Final-B4_0_1-converted 

In [ ]:
model_path = "C:\\Users\\user\\Documents\\Machine-Learning-Hub-1\\Tensorflow\\kaggle_converted\\Final-B4_0_1-converted.keras"

In [ ]:
sm = tf.keras.models.load_model(model_path)

## Vehicle (Tricycle)

In [ ]:
img_path = "D:\DATASETS\Tricycle.JPG"

### Classify Vehicle (Tricycle)

In [ ]:
img_sample = image_to_numpy.load_image_file(os.path.join(img_path))
img_sample = cv2.resize(img_sample, (256, 256))
img_sample = img_sample.astype(np.float32) / 255.0

In [ ]:
yhat = sm.predict(np.expand_dims(img_sample, 0))
print(f"Raw: {yhat}")
print("Labels:   0-Not Road   1-Road")
print(f"Prediction: {np.argmax(yhat)}")

### Plot Vehicle (Tricycle)

In [ ]:
fm_viz(sm, img_path)

## Road (Puddle)

In [ ]:
img_path = "D:\\DATASETS\\For-Thesis\\Excess\\Road\\IMG_20240221_144854.jpg"

### Classify Vehicle (Tricycle)

In [ ]:
img_sample = image_to_numpy.load_image_file(os.path.join(img_path))
img_sample = cv2.resize(img_sample, (256, 256))
img_sample = img_sample.astype(np.float32) / 255.0

In [ ]:
yhat = sm.predict(np.expand_dims(img_sample, 0))
print(f"Raw: {yhat}")
print("Labels:   0-Not Road   1-Road")
print(f"Prediction: {np.argmax(yhat)}")

### Plot Vehicle (Tricycle)

In [ ]:
fm_viz(sm, img_path)

## Road (Pedestrian Lane)

In [ ]:
img_path = "D:\\DATASETS\\For-Issue-Classifier\\Final Classes\\Issue\\IMG_20240223_151958.jpg"

### Classify Vehicle (Tricycle)

In [ ]:
img_sample = image_to_numpy.load_image_file(os.path.join(img_path))
img_sample = cv2.resize(img_sample, (256, 256))
img_sample = img_sample.astype(np.float32) / 255.0

In [ ]:
yhat = sm.predict(np.expand_dims(img_sample, 0))
print(f"Raw: {yhat}")
print("Labels:   0-Not Road   1-Road")
print(f"Prediction: {np.argmax(yhat)}")

### Plot Vehicle (Tricycle)

In [ ]:
fm_viz(sm, img_path)

## Road (Issue Near ADNU)

In [ ]:
img_path = "D:\\DATASETS\\For-Issue-Classifier\\Original Dataset\\Issues\\IMG_7938.jpg"

### Classify Vehicle (Tricycle)

In [ ]:
img_sample = image_to_numpy.load_image_file(os.path.join(img_path))
img_sample = cv2.resize(img_sample, (256, 256))
img_sample = img_sample.astype(np.float32) / 255.0

In [ ]:
yhat = sm.predict(np.expand_dims(img_sample, 0))
print(f"Raw: {yhat}")
print("Labels:   0-Not Road   1-Road")
print(f"Prediction: {np.argmax(yhat)}")

### Plot Vehicle (Tricycle)

In [ ]:
fm_viz(sm, img_path)

## Building (Some Building I Saw)

In [ ]:
img_path = "D:\\DATASETS\\For-Thesis\\Excess\\Building\\IMG_20240226_134825.jpg"

### Classify Vehicle (Tricycle)

In [ ]:
img_sample = image_to_numpy.load_image_file(os.path.join(img_path))
img_sample = cv2.resize(img_sample, (256, 256))
img_sample = img_sample.astype(np.float32) / 255.0

In [ ]:
yhat = sm.predict(np.expand_dims(img_sample, 0))
print(f"Raw: {yhat}")
print("Labels:   0-Not Road   1-Road")
print(f"Prediction: {np.argmax(yhat)}")

### Plot Vehicle (Tricycle)

In [ ]:
fm_viz(sm, img_path)

## Plant ()

In [ ]:
img_path = "D:\\DATASETS\\For-Thesis\\Excess\\Plant\\IMG_20240222_144659.jpg"

### Classify Vehicle (Tricycle)

In [ ]:
img_sample = image_to_numpy.load_image_file(os.path.join(img_path))
img_sample = cv2.resize(img_sample, (256, 256))
img_sample = img_sample.astype(np.float32) / 255.0

In [ ]:
yhat = sm.predict(np.expand_dims(img_sample, 0))
print(f"Raw: {yhat}")
print("Labels:   0-Not Road   1-Road")
print(f"Prediction: {np.argmax(yhat)}")

### Plot Vehicle (Tricycle)

In [ ]:
fm_viz(sm, img_path)

## Vehicle (Zeke's Car)

In [ ]:
img_path = "D:\\DATASETS\\Zeke_Car.jpg"

### Classify Vehicle (Tricycle)

In [ ]:
img_sample = image_to_numpy.load_image_file(os.path.join(img_path))
img_sample = cv2.resize(img_sample, (256, 256))
img_sample = img_sample.astype(np.float32) / 255.0

In [ ]:
yhat = sm.predict(np.expand_dims(img_sample, 0))
print(f"Raw: {yhat}")
print("Labels:   0-Not Road   1-Road")
print(f"Prediction: {np.argmax(yhat)}")

### Plot Vehicle (Tricycle)

In [ ]:
fm_viz(sm, img_path)

## Vehicle (Viola's Motorcycle)

In [ ]:
img_path = "D:\\DATASETS\\Viola_Motor_Zoom.jpg"

### Classify Vehicle (Tricycle)

In [ ]:
img_sample = image_to_numpy.load_image_file(os.path.join(img_path))
img_sample = cv2.resize(img_sample, (256, 256))
img_sample = img_sample.astype(np.float32) / 255.0

In [ ]:
yhat = sm.predict(np.expand_dims(img_sample, 0))
print(f"Raw: {yhat}")
print("Labels:   0-Not Road   1-Road")
print(f"Prediction: {np.argmax(yhat)}")

### Plot Vehicle (Tricycle)

In [ ]:
fm_viz(sm, img_path)

## Vehicle (Zeke Car On Park)

In [ ]:
img_path = "D:\\DATASETS\\Zeke_Car_Park.jpg"

### Classify Vehicle (Tricycle)

In [ ]:
img_sample = image_to_numpy.load_image_file(os.path.join(img_path))
img_sample = cv2.resize(img_sample, (256, 256))
img_sample = img_sample.astype(np.float32) / 255.0

In [ ]:
yhat = sm.predict(np.expand_dims(img_sample, 0))
print(f"Raw: {yhat}")
print("Labels:   0-Not Road   1-Road")
print(f"Prediction: {np.argmax(yhat)}")

### Plot Vehicle (Tricycle)

In [ ]:
fm_viz(sm, img_path)